# Import Library

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import sqlite3
import requests
from bs4 import BeautifulSoup

# Extract Data

In [46]:
# From csv
electricity_access_precent = pd.read_csv('DataSource/electricity_access_precent.csv', quotechar='"')

rural_population_percent = pd.read_csv("DataSource/rural_population_percent.csv", quotechar='"')

gdp_data = pd.read_csv('DataSource/gdp_data.csv', quotechar='"')

mystery = pd.read_csv('DataSource/mystery.csv', quotechar='"')

population_data_from_csv = pd.read_csv('DataSource/population_data.csv', quotechar='"')

projects_data = pd.read_csv("DataSource/projects_data.csv", quotechar='"')


# From json
population_data_from_json = pd.read_json('DataSource/population_data.json')


# From db
conn = sqlite3.connect('DataSource/population_data.db')
population_data_from_db = pd.read_sql_query("SELECT * FROM population_data;", conn)
conn.close()


# From xml
with open("DataSource/population_data.xml", "r") as f:
    xml_data = f.read()

soup = BeautifulSoup(xml_data, "xml")
data = []
for record in soup.find_all("record"):
    record_data = {}
    for field in record.find_all("field"):
        name = field.get("name")
        value = field.text
        record_data[name] = value
    data.append(record_data)

population_data_from_xml = pd.DataFrame(data)


# from API
url = f"https://api.worldbank.org/v2/countries/all/indicators/SP.POP.TOTL/?format=json&per_page=1000"
data = []
page = 1
total_pages = None

while total_pages is None or page <= total_pages:
    response = requests.get(url, params={"page": page})
    response_data = response.json()
    if total_pages is None:
        total_pages = response_data[0]['pages']
    data.extend(response_data[1])
    page += 1

df_worldbank = pd.json_normalize(data)

C:\Users\SINTA\AppData\Local\Temp\ipykernel_3060\3782938366.py:12: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  projects_data = pd.read_csv("DataSource/projects_data.csv", quotechar='"')


# Transfrom

In [47]:
electricity_access_precent.drop(columns=["Unnamed: 62"], inplace=True)
electricity_access_precent = electricity_access_precent.fillna(0.000)
electricity_access_precent.drop(columns=["Indicator Name","Indicator Code"], inplace=True)
electricity_access_precent = electricity_access_precent.rename(columns={'Country Name':'country_name','Country Code':'country_code'})
electricity_access_precent = electricity_access_precent.melt(id_vars=["country_name","country_code"], var_name='year', value_name='electricity_access_percent')

rural_population_percent.drop(columns=["Unnamed: 62"], inplace=True)
rural_population_percent = rural_population_percent.fillna(0.000)
rural_population_percent.drop(columns=["Indicator Name","Indicator Code"], inplace=True)
rural_population_percent = rural_population_percent.rename(columns={'Country Name':'country_name','Country Code':'country_code'})
rural_population_percent = rural_population_percent.melt(id_vars=["country_name","country_code"], var_name='year', value_name='rural_population_percent')

combine = pd.merge(electricity_access_precent, rural_population_percent, how='left', on=["country_name", "country_code", "year"])


In [53]:
# electricity_access_precent.to_csv("tes2.cvs")
# rural_population_percent
# electricity_access_precent
# print(electricity_access_precent.dtypes)
# print(rural_population_percent.dtypes)
# combine

# combine2 = pd.merge(electricity_access_precent, rural_population_percent, how='left', on=["country_name", "country_code", "year"])
# combine2.to_csv('combine2.csv')

In [3]:
# Handle missing value
# Handle duplikat
# Merge data rural_population_percent + electricity_access_percent